# NLP Capstone - Task 3: N-Gram Modeling
Este notebook constrói um modelo preditivo simples baseado em n-gramas (Markov model) para prever a próxima palavra, com técnicas de smoothing e backoff para lidar com n-gramas não vistos no corpus.

In [1]:
import re
import random
from collections import defaultdict, Counter
from nltk.util import ngrams

# Tokenização simples
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

ModuleNotFoundError: No module named 'nltk'

In [ ]:
# Leitura de amostra de texto
def sample_lines(file_path, sample_size=10000, prob=0.03):
    sampled = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            if len(sampled) >= sample_size:
                break
            if random.random() < prob:
                sampled.append(line.strip())
    return sampled

file_path = "/Users/lpn/Downloads/final/en_US/en_US.twitter.txt"
lines = sample_lines(file_path)
tokens = []
for line in lines:
    tokens.extend(tokenize(line))

In [ ]:
# Construção de modelos de 1, 2 e 3-gramas
unigram_model = Counter(tokens)
bigram_model = Counter(ngrams(tokens, 2))
trigram_model = Counter(ngrams(tokens, 3))

In [ ]:
def predict_next_word(input_text, n=3):
    input_tokens = tokenize(input_text)
    if len(input_tokens) < 1:
        return unigram_model.most_common(1)[0][0]

    if n >= 3 and len(input_tokens) >= 2:
        trigram_context = tuple(input_tokens[-2:])
        candidates = {k[2]: v for k, v in trigram_model.items() if k[:2] == trigram_context}
        if candidates:
            return max(candidates, key=candidates.get)

    if n >= 2 and len(input_tokens) >= 1:
        bigram_context = tuple(input_tokens[-1:])
        candidates = {k[1]: v for k, v in bigram_model.items() if k[:1] == bigram_context}
        if candidates:
            return max(candidates, key=candidates.get)

    return unigram_model.most_common(1)[0][0]

In [ ]:
print("Input: i love")
print("Next word prediction:", predict_next_word("i love"))

print("Input: thanks for")
print("Next word prediction:", predict_next_word("thanks for"))

### Discussões:
- **Backoff**: o modelo tenta trigramas, depois bigramas, e por fim unigramas.
- **Smoothing** implícito: toda predição retorna uma palavra, mesmo que não haja contexto prévio.
- **Melhorias possíveis**:
  - Add-1 smoothing (Laplace)
  - Modelo interpolado com pesos
  - Compressão por frequência (remover n-gramas raros)
- **Avaliação**: medir acurácia top-k usando conjuntos de teste e perplexidade